#  Increasing rate of realtime INTEGRAL/IBAS ACS GCN notices 
## Context

We could use more IBAS GCN notices, recovering some unexpectedly missed brighter events as well as some fainter events.

Apart from events missed due to low S/N and verification (excluding short and narrow spikes), nomimal (probably not avoidable) sources of failure to report a burst are:

* telemetry gaps, affecting the burst or background around it
* regions of orbit with (likely) very unstable background, filtered by distance

Many different thresholds are used to find and disciminate the events.  S/N thresholds to report the event in realtime are currently set from 6 to 9, depending on timescale. In addition, thresholds are applied during verification stage.

We would like to change the thresholds to increase number of notices.

It is possible to re-analyse past data with the search, but this process discounts challenges of realtime process, including telemetry interruption effect on the background estimation and burst detection.

To make a live test, S/N thresholds were lowered by 1 sigma (8 to 5), expecting to see more events.Note that here we do not consider another stream of ACS events, using delayed background estimate, and hence higher purity but also larger (few to ten minutes) latency

In [1]:
import re
import requests
from astropy.time import Time
from astropy import units as u
import pandas as pd

In [2]:
# pick real ones

def get_month(month):
    t = requests.get(f"https://www.isdc.unige.ch/integral/ibas/cgi-bin/ibas_acs_web.cgi?month={month}&showall=on").text

    columns = None

    rows = []

    for k in re.findall("<tr.*?</tr>", t, re.S):    

        cs = [c.strip() for c in re.findall("<t[hd].*?>(.*?)</t[hd]>", k, re.S)]

        if columns is None:
            columns = []
            for c in cs:
                c = re.sub("[ \/\[\]]+", "_", c.lower()).strip("_") if 'img' not in c else 'img'
                
                columns.append(c)
        else:
            rows.append(dict(zip(columns, 
                                 [re.sub("\<.*?\>", "", c).strip()  for c in cs])))

        #if columns is None:
        
    return rows

triggers = get_month("2020-09") + get_month("2020-10")

for t in triggers:    
    t['isot'] = "T".join(t['trigger_id_time'].split()[:2])
    t['time'] = Time(t['isot'])
    print(f"{t['origin']:7s} {float(t['sigma']):5.1f} {t['isot']}")

    


AUTO     10.0 2020-09-29T20:44:56
AUTO      9.3 2020-09-28T13:14:40
ASSOC     6.5 2020-09-28T13:14:39
ASSOC     5.0 2020-09-28T06:57:59
ASSOC     6.2 2020-09-25T14:38:02
AUTO      6.5 2020-09-25T11:29:26
AUTO     14.7 2020-09-25T08:09:55
ASSOC     6.1 2020-09-22T17:13:53
ASSOC    15.6 2020-09-20T19:56:59
AUTO      9.8 2020-09-19T23:08:20
ASSOC    10.6 2020-09-19T13:38:51
AUTO      9.2 2020-09-19T13:38:49
AUTO      6.1 2020-09-17T18:10:10
AUTO     11.9 2020-09-17T09:08:56
AUTO     10.1 2020-09-16T19:00:01
AUTO     13.0 2020-09-16T15:41:30
ASSOC     4.9 2020-09-15T03:27:15
OFFLINE  -1.0 2020-09-15T03:27:15
AUTO     10.0 2020-09-14T17:07:03
AUTO      9.3 2020-09-14T17:07:02
ASSOC     9.1 2020-09-14T12:48:30
AUTO      9.0 2020-09-12T14:08:40
ASSOC     5.7 2020-09-09T04:01:22
ASSOC     7.0 2020-09-08T21:15:41
OFFLINE  -1.0 2020-09-07T14:24:20
ASSOC     3.7 2020-09-06T13:11:53
ASSOC     4.4 2020-09-04T02:02:55
AUTO      9.2 2020-09-03T02:34:33
AUTO     10.3 2020-09-03T02:34:31
AUTO      6.3 

In [3]:
# ACS calibration violetly changes the background, and needs to be skipped. it is pre-scheduled and takes ~2 days per year.

skip_calib_range = [Time(c, format="isot") for c in ["2020-09-14T17:15", "2020-09-15T04:00"]]
skip_calib_range

[<Time object: scale='utc' format='isot' value=2020-09-14T17:15:00.000>,
 <Time object: scale='utc' format='isot' value=2020-09-15T04:00:00.000>]

In [4]:
test_log = open("test-so-far").read()
test_triggers = []

I = 0
for r in re.findall(
    #"comment +=.*\n",
    "====== Trigger data ======(.*?)=======",
    test_log,
    re.S,
):
    trigger={k:v for k,v  in re.findall(" ([a-z_]+) += (.*)", r) }        
    
    i = re.search("(\d{4}-\d{2}-\d{2}T\d{2}-\d{2}-\d{2}\.\d+)", r).groups()[0]
    
    d, t = i.split("T")
    i = d+"T"+t.replace("-", ":")    
    it = Time(i, format="isot")
    
    trigger['time'] = it
    
    
    if it.mjd>skip_calib_range[0].mjd and it.mjd<skip_calib_range[1].mjd:
        continue
        
    test_triggers.append(trigger)
            

In [5]:
sc_oor = list(map(lambda t:Time(t).mjd, re.findall('Log_2 +(.*?) ibas.*S/C distance [0-9\.]+ is out of range', test_log)))


In [6]:
vfy_discard = list(map(lambda t:Time(t).mjd, re.findall('Log_2 +(.*?) ibas.*Main_Vfy: Vfy threads say _NO_ burst', test_log)))

len(vfy_discard)

151

In [7]:
fermi_table = pd.read_html(requests.get("https://gcn.gsfc.nasa.gov/fermi_grbs.html").text)[0]

In [8]:
swift_table = pd.read_html(requests.get("https://gcn.gsfc.nasa.gov/swift_grbs.html").text)[0]

In [9]:
acs_table = pd.read_html(requests.get("https://gcn.gsfc.nasa.gov/integral_spiacs.html").text)[0]

In [10]:
fermi_sub_table = pd.read_html(requests.get("https://gcn.gsfc.nasa.gov/gcn/fermi_gbm_subthresh_archive.html").text)[0]


In [11]:
def t2mjd(T, kd, kt):        
    t = "20" + T[kd].str.replace("/","-") + "T" + T[kt]    
    m = t.str.contains('2020-')
    m[m.isna()] = False
    return [Time(_, format='isot').mjd for _ in t[m].unique()]

fermi_mjd = t2mjd(fermi_table, ('TRIGGER', 'Date'), ('TRIGGER', 'Time UT'))
fermi_sub_mjd = t2mjd(fermi_sub_table, ('TRIGGER', 'Date'), ('TRIGGER', 'Time UT'))
swift_mjd = t2mjd(swift_table, ('GRB/TRIGGER', 'Date yy/mm/dd'), ('GRB/TRIGGER', 'Time UT'))
acs_mjd = t2mjd(acs_table, 'Dateyy/mm/dd', 'Time UThh:mm:ss.ss')

In [12]:
one = lambda x:(x[0] if len(x)>0 else {})

S=[]
last_trigger = None
for trigger in sorted([t['time'] for t in triggers] + [t['time'] for t in test_triggers], key=lambda x:x.mjd):
    if trigger < Time("2020-09-09T08:19:29.425"): continue
        
    if last_trigger is not None:
        if (trigger - last_trigger) < 200*u.s:
            continue
        
    last_trigger = trigger
    
    reported = one([ct for ct in triggers if abs(ct['time'].mjd - trigger.mjd)*3600*24<300])
    test_trigger = one([ct for ct in test_triggers if abs(ct['time'].mjd - trigger.mjd)*3600*24<300])    
    
    assoc = []
    anomalies = []
    fixable = []
    
    for table, name, collect in [
        (fermi_mjd, 'Fermi', assoc),
        (fermi_sub_mjd, 'Fermi-sub', assoc),
        (swift_mjd, 'Swift', assoc),
        (acs_mjd, 'ACS-GCN', assoc),
        (sc_oor, 'SC distance', anomalies),
        (vfy_discard, 'Verify Discard', fixable)
    ]:
        if min(abs(table-trigger.mjd))<300./24/3600:
            collect.append(name)                
        
    R = dict(
        time=trigger.isot,
        test_timescale=test_trigger.get('grb_timescale', ''),
        test_grb_sigma=test_trigger.get('grb_sigma', ''),
        reported_comment=reported.get('origin',''),
        reported_sigma=reported.get('sigma', ''),
        assoc='; '.join(assoc),        
        vfy="".join(fixable),
        comment="",
    )
    
    
    if 'ACS-GCN' in assoc:
        if assoc == ['ACS-GCN']:
            R['comment'] = "reported, unconfirmed"
        else:
            R['comment'] = "reported, confirmed"
    else:
        if assoc != []:
            if test_trigger != {}:
                R['comment'] = "recovered"
            else:
                if float(R['reported_sigma'])<8:
                    R['comment'] = "missed, low S/N"
                else:
                    if anomalies != []:
                        R['comment'] = ",".join(anomalies)
                    else:
                        R['comment'] = "MISSED!"
        else:
            if test_trigger != {}:
                R['comment'] = "new unconfirmed"
            

    S.append(R)
    
S = pd.DataFrame(S)
S

,time,test_timescale,test_grb_sigma,reported_comment,reported_sigma,assoc,vfy,comment
0,2020-09-09T08:21:50.275,0.4,5.52067,,,,,new unconfirmed
1,2020-09-09T15:27:14.633,0.1,8.26424,,,,,new unconfirmed
2,2020-09-12T14:08:40.000,0.8,8.85529,AUTO,9.0248,ACS-GCN,,"reported, unconfirmed"
3,2020-09-13T20:04:59.556,0.4,5.20425,,,,,new unconfirmed
4,2020-09-14T01:38:05.172,0.4,5.08649,,,,,new unconfirmed
5,2020-09-14T12:48:30.000,,,ASSOC,9.12,Fermi,Verify Discard,MISSED!
6,2020-09-14T17:07:02.000,2,8.50519,AUTO,10.0079,ACS-GCN,,"reported, unconfirmed"
7,2020-09-15T03:27:15.000,,,ASSOC,4.86,Fermi; Fermi-sub,,"missed, low S/N"
8,2020-09-16T15:41:30.000,0.2,9.95929,AUTO,13.0265,ACS-GCN,,"reported, unconfirmed"
9,2020-09-16T19:00:00.400,2,8.67207,AUTO,10.1447,ACS-GCN,,"reported, unconfirmed"


In [13]:
pd.value_counts(S['comment'])

new unconfirmed          19
reported, unconfirmed     7
missed, low S/N           5
reported, confirmed       3
SC distance               1
MISSED!                   1
Name: comment, dtype: int64

In [14]:
for v, g in S.groupby('comment'):
    try:
        print(f"{v:20s} mean timescale {g['test_timescale'].astype(float).mean():.2f} mean sigma {g['test_grb_sigma'].astype(float).mean():.2f}")
    except (TypeError, ValueError):
        pass

new unconfirmed      mean timescale 0.43 mean sigma 5.77
reported, confirmed  mean timescale 2.00 mean sigma 8.19
reported, unconfirmed mean timescale 0.78 mean sigma 10.21


In [15]:
for v, g in S.groupby('comment'):
    try:
        print(f"{v:20s} mean sigma {g['reported_sigma'].astype(float).mean():.2f}")
    except (TypeError, ValueError):
        pass

MISSED!              mean sigma 9.12
SC distance          mean sigma 15.60
missed, low S/N      mean sigma 5.74
reported, confirmed  mean sigma 9.90
reported, unconfirmed mean sigma 11.27


# Conclusions

Unfortnately no known GBM triggers found by association in the ACS data were recovered with new independent sub-threshold settings. 

This is, however, understandable, since almost all of these GBM events were below also new thresolds.

Two missed events were brighter:

* One was discarded due to orbit phase likely causing variable background.
* One discarded due to noise detection verification. 

This one last event could be probably recovered by adjusting verification thresholds (not touched in this test so far). However, the verification successfully discarded 151 event in the studied interval, and it is understandable if it discards a long weak event in a variable background.

Curiously, not directly related to this test, one of the association detections in this interval was with Fermi/GBM sub-threshold event - confirming it is real. This was supposedly not known without ACS.

Newly produced in real-time sub-threshold events are **largely (but not exclusively) short**.

The event rate is **moderate but sizable, factor 3 higher than current**.

**Increase in reporting of short and possibly hard events, for which ACS has an advantage even over GBM, is desirable**.

Further plans:

* adopt this change into operations
* perhaps make further tests on adjusting other thresholds, possibly on the past data, in order to save the remaining 
